In [1]:
import time, random
from batching import BatchProcessor

In [2]:
def fake_predict(X: list):
    # model accepts a batched input (or list here)

    time.sleep(4)
    n = len(X)
    return [random.choice([0, 1]) for _ in range(n)]


In [3]:
batch_processor = BatchProcessor(fake_predict, bs=16, timeout=1)

`batch_processor.process` puts the input into a queue and returns a future object that can be used to get the result when it's ready.

In [4]:
input = 1
future = batch_processor.process(input)
print(future)

WaitedOjb({'item': 1, 'completed': False, 'result': None, 'completion_time': 'Waiting'})


In [5]:
%%time
result = future.get()
print(result)

1
CPU times: user 854 µs, sys: 743 µs, total: 1.6 ms
Wall time: 5 s


`batch_processor` processes the concurrent requests in batches of 16 (max batch size).

We should get the result for 16 requests under 4 seconds, `fake_predict` takes 4 seconds per request.

In [6]:
num_requests = 16
futures = []
for i in range(num_requests):
    futures.append(batch_processor.process(i))


In [7]:
%%time
for future in futures:
    result = future.get(timeout=10)
    print(result)

1
0
0
0
0
0
0
1
0
1
1
1
1
0
0
0
CPU times: user 2.2 ms, sys: 1.62 ms, total: 3.82 ms
Wall time: 4.99 s
